Calculate taus:

on unit level:
1. Pearsonr trial average
2. STTC trial average
3. STTC trial concat
   
on trial level (not calculated yet):
1. Pearsonr per trial
2. ACF proper per trial
3. iSTTC per trial

In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

In [6]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [7]:
empty_suffix = 'no'
data_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'

### Functions

In [14]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    """
    Calculate tau values per unit using single exponential fitting.

    Parameters:
    - acf_df (pd.DataFrame): DataFrame containing autocorrelation function (ACF) values.
    - acf_cols (list): List of column names containing ACF data.
    - start_idx (int): Starting index for fitting.

    Returns:
    - pd.DataFrame: DataFrame with fitting parameters and tau values per unit.
    """
    acf_2d = acf_df_[acf_cols_].values.astype(np.float64)
    n_units = acf_2d.shape[0]
    print(f'Calculating taus for {acf_2d.shape}')
    
    results = [fit_single_exp(acf_2d[i, :], start_idx_, func_single_exp_monkey) for i in range(n_units)]

    fit_popt_a, fit_popt_b, fit_popt_c, fit_tau, tau_ci_lower, tau_ci_upper, fit_r_squared, explained_var, fit_log_message = zip(*[
        (fit_popt[0] if isinstance(fit_popt, np.ndarray) else np.nan,
         fit_popt[1] if isinstance(fit_popt, np.ndarray) else np.nan,
         fit_popt[2] if isinstance(fit_popt, np.ndarray) else np.nan,
         tau, tau_ci[0], tau_ci[1], fit_r_squared, explained_var, log_msg)
        for fit_popt, _, tau, tau_ci, fit_r_squared, explained_var, log_msg in results
    ])

    # fit_popt, fit_pcov, tau, tau_ci, fit_r_squared, explained_var, log_message

    tau_df = pd.DataFrame({
        'unit_id': acf_df_['unit_id'].values,
        'fit_a': fit_popt_a,
        'fit_b': fit_popt_b,
        'fit_c': fit_popt_c,
        'tau': fit_tau,
        'tau_ci_lower': tau_ci_lower,
        'tau_ci_upper': tau_ci_upper,
        'r_squared': fit_r_squared,
        'explained_var': explained_var,
        'log_message': fit_log_message
    })

    float_cols = ['fit_a', 'fit_b', 'fit_c', 'tau', 'r_squared','explained_var', 'tau_ci_lower', 'tau_ci_upper']
    tau_df[float_cols] = tau_df[float_cols].astype(float)
    
    return tau_df

### Calculate tau

#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [9]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [19]:
area = 'pfdl' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_lower_ms'] = tau_pearsonr_trial_avg_df['tau_ci_lower'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_upper_ms'] = tau_pearsonr_trial_avg_df['tau_ci_upper'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df['tau_ci_lower_ms'] = tau_sttc_trial_avg_df['tau_ci_lower'] * bin_size
tau_sttc_trial_avg_df['tau_ci_upper_ms'] = tau_sttc_trial_avg_df['tau_ci_upper'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df['tau_ci_lower_ms'] = tau_sttc_trial_concat_df['tau_ci_lower'] * bin_size
tau_sttc_trial_concat_df['tau_ci_upper_ms'] = tau_sttc_trial_concat_df['tau_ci_upper'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

Calculating taus for (539, 20)
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf 

In [28]:
tau_sttc_trial_avg_df

,unit_id,fit_a,fit_b,fit_c,tau,tau_ci_lower,tau_ci_upper,r_squared,explained_var,log_message,tau_ms,tau_ci_lower_ms,tau_ci_upper_ms
0,0,0.506156,17.215169,-0.307676,17.215169,-3.592008,38.022347,0.909495,0.909495,ok,860.758472,-179.600422,1901.117367
1,1,0.094437,24.765027,-0.816900,24.765027,-333.823036,383.353090,0.123333,0.123333,ok,1238.251336,-16691.151806,19167.654478
2,2,0.138842,0.573731,-0.014887,0.573731,-12.137981,13.285444,0.005730,0.005730,ok,28.686571,-606.899055,664.272197
3,3,0.237402,7.009008,0.526364,7.009008,3.113288,10.904728,0.910882,0.910882,ok,350.450396,155.664394,545.236399
4,4,0.156735,3.752469,-0.045313,3.752469,-0.260158,7.765095,0.584113,0.584113,ok,187.623437,-13.007884,388.254758
...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,539,0.211762,4.036824,0.195305,4.036824,-2.072918,10.146567,0.425317,0.425317,ok,201.841211,-103.645906,507.328328
535,540,0.341258,1.990948,-0.163937,1.990948,0.719622,3.262274,0.761931,0.761931,ok,99.547394,35.981090,163.113698
536,541,0.641853,2.051229,0.003317,2.051229,1.444164,2.658294,0.937158,0.937158,ok,102.561471,72.208224,132.914717
537,542,0.302688,2.928117,-0.160552,2.928117,0.617339,5.238895,0.692949,0.692949,ok,146.405855,30.866963,261.944747


In [27]:
tau_pearsonr_trial_avg_df

,unit_id,fit_a,fit_b,fit_c,tau,tau_ci_lower,tau_ci_upper,r_squared,explained_var,log_message,tau_ms,tau_ci_lower_ms,tau_ci_upper_ms
0,0,0.719056,19.290684,-0.347091,19.290684,-6.294044,44.875412,0.911963,0.911963,ok,964.534209,-314.702196,2243.770614
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ValueError,NaN,NaN,NaN
2,2,0.110822,0.994330,0.019220,0.994330,-8.054737,10.043397,0.018581,0.018581,ok,49.716504,-402.736854,502.169862
3,3,0.279456,6.085777,0.663298,6.085777,2.853887,9.317667,0.902339,0.902339,ok,304.288860,142.694368,465.883353
4,4,0.194176,3.603661,-0.088259,3.603661,0.558895,6.648427,0.686497,0.686497,ok,180.183037,27.944734,332.421341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,539,0.200109,7.218432,0.110975,7.218432,-6.982118,21.418982,0.459831,0.459831,ok,360.921577,-349.105924,1070.949078
535,540,0.475257,2.092072,-0.143533,2.092072,1.025133,3.159010,0.834083,0.834083,ok,104.603578,51.256647,157.950510
536,541,0.809778,2.148375,-0.022680,2.148375,1.503493,2.793256,0.935643,0.935643,ok,107.418727,75.174664,139.662790
537,542,0.277749,2.963283,-0.157117,2.963283,0.397074,5.529491,0.653134,0.653134,ok,148.164128,19.853704,276.474552


In [29]:
area = 'pfp' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_lower_ms'] = tau_pearsonr_trial_avg_df['tau_ci_lower'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_upper_ms'] = tau_pearsonr_trial_avg_df['tau_ci_upper'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df['tau_ci_lower_ms'] = tau_sttc_trial_avg_df['tau_ci_lower'] * bin_size
tau_sttc_trial_avg_df['tau_ci_upper_ms'] = tau_sttc_trial_avg_df['tau_ci_upper'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df['tau_ci_lower_ms'] = tau_sttc_trial_concat_df['tau_ci_lower'] * bin_size
tau_sttc_trial_concat_df['tau_ci_upper_ms'] = tau_sttc_trial_concat_df['tau_ci_upper'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

Calculating taus for (538, 20)
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf 

FileNotFoundError: [Errno 2] No such file or directory: 'Q:\\Personal\\Irina\\projects\\isttc\\results\\monkey\\fixation_period_1000ms_no_empty\\non_binned\\pfp\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl'

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

In [ ]:
# n_lags = 20
# bin_size = 50
# acf_cols = ['acf_' + str(i) for i in range(n_lags)]
# print('acf_cols {}'.format(acf_cols))

In [ ]:
# area = 'pfdl' 

# pearsonr 
# pearsonr_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(pearsonr_per_trial_df)}, dropping before tau calc ...')
# pearsonr_per_trial_df.dropna(inplace=True)
# pearsonr_per_trial_df.reset_index(drop=True, inplace=True)
# tau_pearsonr_per_trial_df = calc_tau_per_unit(pearsonr_per_trial_df, acf_cols[:-1], start_idx_=1)
# tau_pearsonr_per_trial_df['tau_ms'] = tau_pearsonr_per_trial_df['tau'] * bin_size
# tau_pearsonr_per_trial_df.insert(1, 'trial_id', pearsonr_per_trial_df['trial_id'])
# tau_pearsonr_per_trial_df.insert(2, 'condition_id', pearsonr_per_trial_df['condition_id'])
# tau_pearsonr_per_trial_df.insert(3, 'spike_count', pearsonr_per_trial_df['spike_count'])
# tau_pearsonr_per_trial_df.insert(4, 'fr_hz', pearsonr_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_pearsonr_per_trial_df)}')
# tau_pearsonr_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # acf formula
# pcf_proper_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pcf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(pcf_proper_per_trial_df)}, dropping before tau calc ...')
# pcf_proper_per_trial_df.dropna(inplace=True)
# pcf_proper_per_trial_df.reset_index(drop=True, inplace=True)
# tau_acf_proper_per_trial_df = calc_tau_per_unit(pcf_proper_per_trial_df, acf_cols, start_idx_=1)
# tau_acf_proper_per_trial_df['tau_ms'] = tau_acf_proper_per_trial_df['tau'] * bin_size
# tau_acf_proper_per_trial_df.insert(1, 'trial_id', pcf_proper_per_trial_df['trial_id'])
# tau_acf_proper_per_trial_df.insert(2, 'condition_id', pcf_proper_per_trial_df['condition_id'])
# tau_acf_proper_per_trial_df.insert(3, 'spike_count', pcf_proper_per_trial_df['spike_count'])
# tau_acf_proper_per_trial_df.insert(4, 'fr_hz', pcf_proper_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_acf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_acf_proper_per_trial_df)}')
# tau_acf_proper_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # isttc
# sttc_per_trial_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {sttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(sttc_per_trial_df)}, dropping before tau calc ...')
# sttc_per_trial_df.dropna(inplace=True)
# sttc_per_trial_df.reset_index(drop=True, inplace=True)
# tau_isttc_per_trial_df = calc_tau_per_unit(sttc_per_trial_df, acf_cols, start_idx_=1)
# tau_isttc_per_trial_df['tau_ms'] = tau_isttc_per_trial_df['tau'] * bin_size
# tau_isttc_per_trial_df.insert(1, 'trial_id', sttc_per_trial_df['trial_id'])
# tau_isttc_per_trial_df.insert(2, 'condition_id', sttc_per_trial_df['condition_id'])
# tau_isttc_per_trial_df.insert(3, 'spike_count', sttc_per_trial_df['spike_count'])
# tau_isttc_per_trial_df.insert(4, 'fr_hz', sttc_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_isttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_isttc_per_trial_df)}')
# tau_isttc_per_trial_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')